In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the dataset
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/datasets/Air_Qualitydatasetcsv.csv') # Assuming the file is in this path



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Create a mapping from Geo Place Name to Geo Join ID
geo_mapping = data[['Geo Place Name', 'Geo Join ID']].drop_duplicates().set_index('Geo Place Name').to_dict()['Geo Join ID']

# Function to preprocess user input
def preprocess_input(start_date, geo_place_name):
    # Extract the year from Start_Date
    year = int(pd.to_datetime(start_date).year)

    # Get Geo Join ID from Geo Place Name
    geo_join_id = geo_mapping.get(geo_place_name, None)
    if geo_join_id is None:
        raise ValueError(f"Geo Place Name '{geo_place_name}' not found in the dataset.")

    # Create a feature vector
    features = np.array([[geo_join_id, year]])

    # Normalize the features (using the same scaler as during training)
    scaler = StandardScaler()
    X_train = data[['Geo Join ID', 'Time Period']]
    X_train['Time Period'] = X_train['Time Period'].apply(lambda x: int(x.split()[-1]))
    scaler.fit(X_train)  # Fit the scaler on the training data
    features_scaled = scaler.transform(features)

    # Add a column of ones for the bias term
    features_scaled = np.c_[np.ones(features_scaled.shape[0]), features_scaled]

    return features_scaled

# Function to predict Data Value
def predict_data_value(theta, start_date, geo_place_name):
    # Preprocess the input
    features = preprocess_input(start_date, geo_place_name)

    # Make prediction
    prediction = features.dot(theta)
    return prediction[0]


In [ ]:
# Example usage
if __name__ == "__main__":
    # Trained model parameters (replace with your actual theta values)
    theta = np.array([0.5, 0.3, -0.2])  # Example values

    # Get user input
    start_date = input("Enter Start Date (MM/DD/YYYY): ")
    geo_place_name = input("Enter Geo Place Name: ")

    # Predict Data Value
    try:
        prediction = predict_data_value(theta, start_date, geo_place_name)
        print(f"Predicted Data Value for {geo_place_name} on {start_date}: {prediction:.2f}")
    except ValueError as e:
        print(e)

Enter Start Date (MM/DD/YYYY): 01/01/2005
Enter Geo Place Name: East New York
Predicted Data Value for East New York on 01/01/2005: 1.09


<ipython-input-16-b5616bef8c41>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Time Period'] = X_train['Time Period'].apply(lambda x: int(x.split()[-1]))
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
